In [1]:
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException 
import re
import os

In [113]:


url="https://www.oddsportal.com/matches/tennis/"



driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
driver.get(url)
sleep(5)


table=driver.find_elements_by_xpath("//*[@id='table-matches']/table/tbody/tr[contains(@class,'dark')]")

list_href=[]
list_location=[]
list_player1=[]
list_player2=[]
list_odd1=[]
list_odd2=[]
list_winnerP1=[]


for row in table:
    if ("ATP" in row.text) and (not "Doubles" in row.text):
        href=row.find_element_by_xpath('./th[1]/a[2]').get_attribute('href')
        if href not in list_href:
            list_href.append(href)
driver.quit()


for href in list_href:
    
    driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
    driver.get(href)
    sleep(10)
    
    h1=driver.find_element_by_xpath('//*[@id="col-content"]/h1').text
    location=h1.replace('Betting Odds','')
    location=location.replace('ATP ','')
    location=re.sub(r'\([^)]*\)', '', location)
    location=location.rstrip().lstrip()
    matches=driver.find_elements_by_xpath("//*[@id='tournamentTable']/tbody/tr")
    eventNb=0
    for match in matches:
        #print(match.text)
        try:
            className=match.get_attribute("class")
            if className=='center nob-border':
                eventNb+=1
            if eventNb==1:
                if not('dark' or 'nob-border' or 'table-dummyrow')in className:
           
                    try:
                        if 'deactivate' in className:
                            players=match.find_element_by_xpath('./td[2]/a').text
                            
                            if players=="":
                                players=match.find_element_by_xpath('./td[2]/a[2]').text
                            print(players)
                            winnerP1=match.find_element_by_xpath('./td[3]').text
                            print(winnerP1)
                            odd1=match.find_element_by_xpath('./td[4]/a').text
                            odd1_className=match.find_element_by_xpath('./td[4]').get_attribute("class")
                            odd2=match.find_element_by_xpath('./td[5]/a').text
                            odd2_className=match.find_element_by_xpath('./td[5]').get_attribute("class")
                            print(odd1_className)
                            print(odd2_className)
                            if ("ret" in winnerP1) or ("canc" in winnerP1):
                                winnerP1=np.nan
                            elif (("result-ok" not in odd1_className) & ("result-ok" not in odd2_className)): #match not finished
                                winnerP1=np.nan
                                print("not in")
                            elif int(winnerP1[0])<int(winnerP1[2]):
                                winnerP1=0
                            else:
                                winnerP1=1
                        else:
                            players=match.find_element_by_xpath('./td[2]/a[not(contains(@class,"ico-tv-tournament"))]').text
                            odd1=match.find_element_by_xpath('./td[3]/a').text
                            odd2=match.find_element_by_xpath('./td[4]/a').text
                            winnerP1=np.nan
                        
                        player1=players.split(" - ")[0]
                        player1=player1.replace("-",".")
                        player2=players.split(" - ")[1]
                        player2=player2.replace("-",".")
                        list_location.append(location)
                        list_player1.append(player1)
                        list_player2.append(player2)
                        list_odd1.append(odd1)
                        list_odd2.append(odd2)
                        list_winnerP1.append(winnerP1)
                    except :
                        continue
                    
        except NoSuchElementException:
            continue
    #driver.quit()
dictionnary={"Location":list_location,"Player1":list_player1,"Player2":list_player2,"AvgP1":list_odd1,"AvgP2":list_odd2,"WinnerP1":list_winnerP1}

            
    

Hanfmann Y. - Cecchinato M.
2:0
odds-nowrp result-ok
odds-nowrp
Evans D. - Musetti L.
1:2
odds-nowrp
odds-nowrp result-ok
Simon G. - Sonego L.
0:2
odds-nowrp
odds-nowrp result-ok
Djere L. - Millman J.
1:0
odds-nowrp
odds-nowrp
not in
Davidovich Fokina A. - Ivashka I.
0:2
odds-nowrp
odds-nowrp result-ok
Munar J. - Fognini F.
2:0
odds-nowrp result-ok
odds-nowrp
Mager G. - Ruud C.
0:2
odds-nowrp
odds-nowrp result-ok
Lopez F. - Alcaraz C.
1:0
odds-nowrp
odds-nowrp
not in


In [107]:
print(dictionnary)

{'Location': ['Cagliari', 'Cagliari', 'Cagliari', 'Cagliari', 'Marbella', 'Marbella', 'Marbella', 'Marbella'], 'Player1': ['Hanfmann Y.', 'Evans D.', 'Simon G.', 'Djere L.', 'Davidovich Fokina A.', 'Munar J.', 'Mager G.', 'Lopez F.'], 'Player2': ['Cecchinato M.', 'Musetti L.', 'Sonego L.', 'Millman J.', 'Ivashka I.', 'Fognini F.', 'Ruud C.', 'Alcaraz C.'], 'AvgP1': ['1.82', '3.68', '3.22', '1.32', '1.25', '1.73', '2.50', '3.32'], 'AvgP2': ['1.99', '1.29', '1.35', '3.44', '3.99', '2.14', '1.53', '1.34'], 'WinnerP1': [nan, nan, nan, nan, nan, nan, nan, nan]}


In [114]:
df_matchs=pd.DataFrame(dictionnary)

In [86]:
locations_list=df_validation['Location'].unique()

In [87]:
print(type(locations_list))

<class 'numpy.ndarray'>


In [75]:
url_tourn="https://www.atptour.com/en/tournaments/"

driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
driver.get(url_tourn)
sleep(5)

month_tournament=driver.find_elements_by_xpath("//*[@id='contentAccordionWrapper']/div[contains(@class,'expand')][1]/div[2]/div/table/tbody/tr")

list_series=[]
list_court=[]
list_location=[]
list_surface=[]

for tournament in month_tournament:
    #print(tournament)
    try:
        location=tournament.find_element_by_xpath('./td[2]/span[1]').text
        town=location.split(",")[0]
        if town in locations_list:
            try:
                series=tournament.find_element_by_xpath('./td[1]/img').get_attribute("src")
                if "250.png" in series:
                    series="ATP250"
                elif "500.png" in series:
                    series="ATP500"
                elif "1000.png" in series:
                    series="Masters 1000"
                elif "grandslam.png" in series:
                    series="Grand Slam"
                elif "finals.svg" in series:
                    series="ATP Finals"
                else:
                    series=np.nan
            except:
                series=np.nan
            try:
                playground=tournament.find_element_by_xpath('./td[3]/table/tbody/tr/td[2]/div/div').text
                playground=playground.split(" ")
                court=playground[0]
                surface=playground[1]
            except:
                court=np.nan
                surface=np.nan
            list_series.append(series)
            list_surface.append(surface)
            list_court.append(court)
            list_location.append(town)
    except:
        print("nan")

driver.quit()
loc_dict={"Location":list_location,"Series":list_series,"Court":list_court,"Surface":list_surface}

In [76]:
print(loc_dict)

{'Location': ['Marbella', 'Cagliari'], 'Series': ['ATP250', 'ATP250'], 'Court': ['Outdoor', 'Outdoor'], 'Surface': ['Clay', 'Clay']}


In [77]:
df_tournament=pd.DataFrame(loc_dict)

In [78]:
df_tournament.head()

,Location,Series,Court,Surface
0,Marbella,ATP250,Outdoor,Clay
1,Cagliari,ATP250,Outdoor,Clay


In [125]:
df_matchs.head(10)

,Location,Player1,Player2,AvgP1,AvgP2,WinnerP1
0,Cagliari,Hanfmann Y.,Cecchinato M.,1.82,1.99,1.0
1,Cagliari,Evans D.,Musetti L.,3.68,1.29,0.0
2,Cagliari,Simon G.,Sonego L.,3.22,1.35,0.0
3,Cagliari,Djere L.,Millman J.,1.32,3.44,NaN
4,Marbella,Davidovich Fokina A.,Ivashka I.,1.25,3.99,0.0
5,Marbella,Munar J.,Fognini F.,1.73,2.14,1.0
6,Marbella,Mager G.,Ruud C.,2.50,1.53,0.0
7,Marbella,Lopez F.,Alcaraz C.,3.32,1.34,NaN


In [126]:
df_validation=df_matchs.merge(df_tournament,on="Location")
df_validation.head(10)

,Location,Player1,Player2,AvgP1,AvgP2,WinnerP1,Series,Court,Surface
0,Cagliari,Hanfmann Y.,Cecchinato M.,1.82,1.99,1.0,ATP250,Outdoor,Clay
1,Cagliari,Evans D.,Musetti L.,3.68,1.29,0.0,ATP250,Outdoor,Clay
2,Cagliari,Simon G.,Sonego L.,3.22,1.35,0.0,ATP250,Outdoor,Clay
3,Cagliari,Djere L.,Millman J.,1.32,3.44,NaN,ATP250,Outdoor,Clay
4,Marbella,Davidovich Fokina A.,Ivashka I.,1.25,3.99,0.0,ATP250,Outdoor,Clay
5,Marbella,Munar J.,Fognini F.,1.73,2.14,1.0,ATP250,Outdoor,Clay
6,Marbella,Mager G.,Ruud C.,2.50,1.53,0.0,ATP250,Outdoor,Clay
7,Marbella,Lopez F.,Alcaraz C.,3.32,1.34,NaN,ATP250,Outdoor,Clay
